In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import os
import shutil
from tensorflow.keras.applications import EfficientNetB0 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization 
from tensorflow.keras.models import Model 

In [3]:
data_dir = "/Users/michelangelozampieri/Desktop/TAMID-Group-New/data/sorted_data"

classes = []
for root, dirs, files in os.walk(data_dir):
    for name in dirs:
        classes.append(name)
print(classes)

['PP', 'Other', 'PE-HD', 'PS', 'PET', 'PVC']


In [4]:
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Initially freeze the base model

# Custom classification head
x = GlobalAveragePooling2D()(base_model.output)  # Reduce feature maps to a single vector
x = Dense(64, activation="relu")(x)  # Fewer neurons for simplicity
x = BatchNormalization()(x)  # Optional: Stabilize training
output_layer = Dense(len(classes), activation="softmax")(x)  # Output layer

# Create model
model = Model(inputs=base_model.input, outputs=output_layer)

In [5]:
train_dir = "/Users/michelangelozampieri/Desktop/TAMID-Group-New/data/sorted_data_output/train"
test_dir = "/Users/michelangelozampieri/Desktop/TAMID-Group-New/data/sorted_data_output/test"
validation_dir = "/Users/michelangelozampieri/Desktop/TAMID-Group-New/data/sorted_data_output/validation"

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

Found 1606 images belonging to 6 classes.


In [7]:
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

Found 351 images belonging to 6 classes.


In [8]:
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels="inferred",  
    label_mode="categorical",  
    image_size=(224, 224)
)

Found 349 files belonging to 6 classes.


In [9]:
model.compile(optimizer="adam", 
              loss="categorical_crossentropy", 
              metrics=["accuracy"])

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20
)

/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 65s 1s/step - accuracy: 0.3009 - loss: 1.8523 - val_accuracy: 0.5499 - val_loss: 1.5784
Epoch 2/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 59s 1s/step - accuracy: 0.5128 - loss: 1.4999 - val_accuracy: 0.5499 - val_loss: 1.4031
Epoch 3/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - accuracy: 0.5503 - loss: 1.3553 - val_accuracy: 0.5499 - val_loss: 1.6350
Epoch 4/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.5444 - loss: 1.3365 - val_accuracy: 0.5499 - val_loss: 1.3594
Epoch 5/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 59s 1s/step - accuracy: 0.5393 - loss: 1.3488 - val_accuracy: 0.5499 - val_loss: 1.5248
Epoch 6/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.5393 - loss: 1.3208 - val_accuracy: 0.5499 - val_loss: 1.3505
Epoch 7/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.5457 - loss: 1.3504 - val_accuracy: 0.5499 - val_loss: 1.3618
Epoch 8/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.5359 - loss: 1.3497 - val_accuracy: 0.5641 - val_loss:

In [11]:
val_loss, val_accuracy = model.evaluate(test_dataset)
print(f"Validation accuracy: {val_accuracy}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 408ms/step - accuracy: 0.2318 - loss: 17.4579
Validation accuracy: 0.21776504814624786


In [12]:
model_dir = "/Users/michelangelozampieri/Desktop/TAMID-group-New/models"

In [13]:
model.save(os.path.join(model_dir, "efficienet_base_model.h5"))

In [15]:
model.trainable = True
for layer in model.layers[:-20]:  # Freeze all layers except the last 20
    layer.trainable = False

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

history_fine = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

Epoch 1/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 63s 1s/step - accuracy: 0.5671 - loss: 1.2990 - val_accuracy: 0.5613 - val_loss: 1.3802
Epoch 2/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.5734 - loss: 1.2861 - val_accuracy: 0.5527 - val_loss: 1.3143
Epoch 3/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.5581 - loss: 1.2850 - val_accuracy: 0.5527 - val_loss: 1.2991
Epoch 4/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.5633 - loss: 1.2969 - val_accuracy: 0.5527 - val_loss: 1.2981
Epoch 5/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - accuracy: 0.5488 - loss: 1.3027 - val_accuracy: 0.5527 - val_loss: 1.2968
Epoch 6/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.5398 - loss: 1.3349 - val_accuracy: 0.5527 - val_loss: 1.2984
Epoch 7/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.5834 - loss: 1.2493 - val_accuracy: 0.5499 - val_loss: 1.2990
Epoch 8/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.5505 - loss: 1.2976 - val_accuracy: 0.5499 - val_loss:

In [16]:
val_loss, val_accuracy = model.evaluate(test_dataset)
print(f"Validation accuracy: {val_accuracy}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 410ms/step - accuracy: 0.2162 - loss: 18.8457
Validation accuracy: 0.22063037753105164


In [17]:
model.save(os.path.join(model_dir, "efficienet_fine_tuned_model.h5"))